## Running Geneland - N5NB


The data use here comes from ddRadseq from 112 samples of Campylorhynchus zonatus and Campylorhynchus fasciatus that occur along the precipitation gradient on the Pacific region of Ecuador. N5NB means that this SNPs data set does not contain index group 5 from the sequencing (low reading in the samples) and the outgroup Campylorhynchus bruneicapillus. This is just a code to indentified the data set I'm working on.

Setting the directory and reading SNPs and coordinates files

In [ ]:
setwd("C:/Users/Owner/Desktop/Dropbox/Thesis/Molecular_Wrens/Wren_1/Geneland")

### Data Management

Reading the SPNs data as txt, this is the structure format from the output of Stack except I manually deleted the first line of credit and the save it as txt in excel.

In [ ]:
wn5nb <- read.table("wn5nb_2.txt", header = TRUE, row.names = NULL)

In [ ]:
We will need the following package to data management.

In [ ]:
install.packages("tidyverse")
library(tidyverse)

I created this function that take the txt file in Structure long format and gives the Geneland format as output.


In [ ]:
struc2geneland <-function(data, nsnp, ploid, samples, pop){
        
        data$id <- rep(1:samples, each=ploid)

        arg1=pop
        arg2=data

        pop.del <- function(arg1, arg2){
                if(arg1==1){
                      x <- arg2[,-c(1,2)]   
               }
                else {
                      x <- arg2[,-1] 
                }
               return(x)
        }

                 output <- pop.del(pop, data)
                 output <-  pivot_longer(output, cols=!id, names_to="marker", values_to="snp")
                 output$dip <- rep(1:nsnp,samples*ploid)
                 output <- output[order(output$id, output$dip),]
                 output$dip <- rep(1:(nsnp*ploid), samples)
                 output <- output[,-2]
                 output <- pivot_wider(output, names_from=dip, values_from=snp)
                 output <- output[,-1]
                 output[output==0] <- NA
        return(output)
}

Now we use the function I created above. We have to write down the parameter for our specific data set.

In [ ]:
wn5nb.2 <- struc2geneland(data=wn5nb, nsnp=4409, ploid=2, samples=112, pop=0)

We can save this geneland format SNPs file in your local computer in case you need it for other analysis.

In [ ]:
write.table(wn5nb.2, "./wn5nb_geneland_hg.txt", row.names=FALSE, col.names=FALSE)

#### Putting Coordinates of Samples in the Same Order as in the SNPs file

We have an issue now. We need the coordinate for each sample or population for a Spatial Explicit model in Geneland. The order of samples has to be the same as the SNPs geneland-format file. So we need to do some data management there. I am going to use the popmap file use in Stack to tell the software the labels of samples and populations. The output files from Stack and our Geneland-format files have the same order. 

In [ ]:
labels <- read.table("popmap_wren_n5nb_sh11.txt", col.names = c("Ind", "ord"))

Adding a column for the order of samples

In [ ]:
labels$ord <- 1:nrow(labels)

I also going to use the meta data of the samples, including coordinates, locations names, etc.


In [ ]:
labels.all <- read.table("Labels.field.csv", header=TRUE, sep=",", na.strings=TRUE, fileEncoding = "UTF-8-BOM")


Merging the data set with coordinates and samples order

In [ ]:
coor <- merge(labels, labels.all, by="Ind")[,c(1,2,9,10)]

Ordering by samples

In [ ]:
coor2 <- coor[order(coor$ord,  decreasing = FALSE), -c(1:2)]

Geneland read Longitude as the firsts collumns. I switch the columns for geneland format and making them numeric.

In [ ]:
coor3 <- cbind(as.numeric(as.character(coor2$Long)), as.numeric(as.character(coor2$Lat)))

We have another issue. Geneland use UTM coordinate system. The delta parameter the function Geneland specifies a buffer around the coordinates that works as noise when the coordinates are not very accurate. For this reason it is better for the software to deal with a planar system. We have to transform the coordinates to UTM.

Installing rgdal to transform coordinates

In [ ]:
install.packages("rgdal")
library(rgdal)

Setting the coordinates system as Long/Lat

In [ ]:
cord.dec = SpatialPoints(cbind(coor3[,1], coor3[,2]), proj4string=CRS("+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84"))

Converting to UTM, I'm using UTM zone 17s (epsg=32717) because most of the territory of Ecuador is in this zone.

In [ ]:
coor.utm <- spTransform(cord.dec, CRS("+init=epsg:32717"))
colnames(coor.utm@coords) <- c("X", "Y")

Write the coordinates to use it in other analyses.

In [ ]:
write.table(coor.utm@coords, "./coord1.txt", row.names=FALSE, col.names=FALSE)

### Running Geneland

Reading the SNPs and coordinates files.

In [ ]:
# Read the file in geneland format
wn5nb <- read.table("./wn5nb_geneland_hg.txt", header = FALSE, row.names = NULL)

# Read the file of coordinates
coord <- read.table("./coord.txt", header = FALSE, row.names = NULL)

Installing Geneland: first we have to install MiKTeX in your local computer and devtools. To install MiKTeX go to https://miktex.org/download and follow the instructions.

Then install devtools

In [ ]:
install.packages("devtools")
library(devtools)

Install Geneland

In [ ]:
if( ! 'devtools' %in% installed.packages() )  { install.packages('devtools') }
devtools::install_github('gilles-guillot/Geneland', build_vignettes = TRUE)

In [ ]:
# Installing Geneland
library(Geneland)

Calling parallel packages, not need to install. It comes with base

In [ ]:
library(parallel)

Creating some vectors for the burnins, runs and Ks. You can run more replicates and Ks by modifying the vectors below. Before modifying the vectors, be sure you have enough computing power. 

In [ ]:
nrun <- 1:30
burnin <- 200
ks <- 2:15

In the next lines, I create directories for each K and run (replicate). Each K will run in a separate core in your computer. You need to be sure you have enough cores for your code. If you are not running this code in a hpc (high performace computing) like hipergator, you can figure out how many cores do you have by running detectCores() after calling the parallel package.

There a lot of techniques to run functions in parallel. I experience some issues using Foreach and DoParallel in Linux when I tried to ran MCMC in the hpc. In case you are running MCMC in Windows, you can try these methods.

In [ ]:
parallel::mclapply(1:length(ks), function(g, n, i, r) {
  for(r in 1:length(nrun)){
  path.mcmc <- paste("./Output/", "k", g[i], "r", n[r],  "/", sep="")
  dir.create(path=path.mcmc)
  Geneland::MCMC(coordinates=coord,
                 geno.dip.codom=wn5nb,
                 varnpop=TRUE,
                 npopmax=g[i],
                 spatial=TRUE,
                 freq.model="Correlated",
                 delta.coord=1000,
                 nit=500000,
                 thinning=100,
                 path.mcmc=path.mcmc)
  
  ## MCMC postprocessing
  PostProcessChain(coordinates=coord,
                   path.mcmc=path.mcmc,
                   nxdom=200,
                   nydom=200,
                   burnin=burnin)
  }
}, n=nrun, g=ks, mc.cores=16)